In [1]:
%cd /content/drive/MyDrive/inference_SAM_video

/content/drive/MyDrive/inference_SAM_video


In [ ]:
!git clone https://github.com/facebookresearch/segment-anything-2.git


Cloning into 'segment-anything-2'...
remote: Enumerating objects: 510, done.
remote: Counting objects: 100% (198/198), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 510 (delta 132), reused 74 (delta 50), pack-reused 312 (from 1)
Receiving objects: 100% (510/510), 78.53 MiB | 12.99 MiB/s, done.
Resolving deltas: 100% (149/149), done.
Updating files: 100% (263/263), done.


In [2]:
%cd segment-anything-2
!pip install -e .

/content/drive/MyDrive/inference_SAM_video/segment-anything-2
Obtaining file:///content/drive/MyDrive/inference_SAM_video/segment-anything-2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cubl

In [7]:
from sam2.build_sam import build_sam2_video_predictor

sam2_checkpoint = "/content/drive/MyDrive/inference_SAM_video/ckpt/sam2_hiera_large.pt"
model_cfg = "sam2_hiera_l.yaml"
device = "cuda"
sam2predictor = build_sam2_video_predictor(model_cfg, sam2_checkpoint, device=device)

In [8]:
import numpy as np
def compute_iou(pred_mask, gt_semantic_seg):
    in_mask = np.logical_and(gt_semantic_seg, pred_mask)
    out_mask = np.logical_or(gt_semantic_seg, pred_mask)
    iou = np.sum(in_mask) / np.sum(out_mask)
    return iou
def compute_dice(mask_gt, mask_pred):
    """Compute soerensen-dice coefficient.
    Returns:
    the dice coeffcient as float. If both masks are empty, the result is NaN
    """
    volume_sum = mask_gt.sum() + mask_pred.sum()
    if volume_sum == 0:
        return np.NaN
    volume_intersect = (mask_gt & mask_pred).sum()
    return 2*volume_intersect / volume_sum

In [23]:
import os
import cv2
resultpath = '/content/drive/MyDrive/inference_SAM_video/result/polypgen/sam2_pointpernframe'
datapath = '/content/drive/MyDrive/inference_SAM_video/data/polypgen'
promptpath = '/content/drive/MyDrive/inference_SAM_video/prompt/point/polypgen'
visualize = '/content/drive/MyDrive/inference_SAM_video/visresult'
# classes = ['bg_change','blood','low_brightness','regular','smoke']
# direction = ['left','right']
for i in os.listdir(datapath):
  count = 0
  inference_state = sam2predictor.init_state(os.path.join(datapath,i,'images'))
  sam2predictor.reset_state(inference_state)
  for j in range(len(os.listdir(os.path.join(datapath,i,'masks')))):
      # if os.path.exists(os.path.join(resultpath,i,j,k,l.split('.')[0],'dice.npy')):
      #   continue
      print(os.path.join(datapath,i,'masks',str(j)))
      positive_points = np.load(os.path.join(promptpath,i.split('.nii.gz')[0],str(count),'positive.npy'))
      negative_points = np.load(os.path.join(promptpath,i.split('.nii.gz')[0],str(count),'negative.npy'))
      input_points = np.concatenate([positive_points, negative_points])
      for m in range(len(input_points)):
        a = input_points[m][0]
        input_points[m][0] = input_points[m][1]
        input_points[m][1] = a
      input_labels = np.concatenate([np.ones(positive_points.shape[0]), np.zeros(negative_points.shape[0])])
      if (len(positive_points) == 0):
        count += 1
        continue
      _, out_obj_ids, out_mask_logits = sam2predictor.add_new_points_or_box(
        inference_state=inference_state,
        frame_idx=count,
        obj_id=1,
        points=input_points,
        labels=input_labels,
          )

      mask = (out_mask_logits[0] > 0.0).cpu().numpy()
      gt = np.load(os.path.join(datapath,i,'binarymask',str(j) + '.npy'))
      dice = []
      iou = []
      count += 1
      os.makedirs(os.path.join(resultpath,i,str(j)),exist_ok=True)
      np.save(os.path.join(resultpath,i,str(j),'mask' + str(m)+'.npy'),mask)
      dice.append(compute_dice(gt,mask))
        # iou.append(compute_iou(masks[m],gt))
      print(dice)
      d = np.argsort(dice)
      np.save(os.path.join(resultpath,i,str(j),'dice.npy'),dice)
      # np.save(os.path.join(resultpath,i,j,k,l.split('.')[0],'iou.npy'),iou)

frame loading (JPEG): 100%|██████████| 249/249 [00:12<00:00, 20.06it/s]


/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/0
[0.9727428971760069]
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/1
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/2
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/3
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/4
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/5
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/6
<generator object SAM2VideoPredictor.propagate_in_video at 0x786dd916a570>


/content/drive/MyDrive/inference_SAM_video/segment-anything-2/sam2/sam2_video_predictor.py:873: UserWarning: /content/drive/MyDrive/inference_SAM_video/segment-anything-2/sam2/_C.so: undefined symbol: _ZN3c1015SmallVectorBaseIjE8grow_podEPKvmm

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/segment-anything-2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(


TypeError: 'int' object is not subscriptable

In [24]:
import os
import cv2
resultpath = '/content/drive/MyDrive/inference_SAM_video/result/polypgen/sam2_pointpernframe'
datapath = '/content/drive/MyDrive/inference_SAM_video/data/polypgen'
promptpath = '/content/drive/MyDrive/inference_SAM_video/prompt/point/polypgen'
visualize = '/content/drive/MyDrive/inference_SAM_video/visresult'
# classes = ['bg_change','blood','low_brightness','regular','smoke']
# direction = ['left','right']
for i in os.listdir(datapath):
  count = 0
  c = 0
  inference_state = sam2predictor.init_state(os.path.join(datapath,i,'images'))
  sam2predictor.reset_state(inference_state)
  for j in range(len(os.listdir(os.path.join(datapath,i,'masks')))):
      # if os.path.exists(os.path.join(resultpath,i,j,k,l.split('.')[0],'dice.npy')):
      #   continue
      print(os.path.join(datapath,i,'masks',str(j)))
      positive_points = np.load(os.path.join(promptpath,i.split('.nii.gz')[0],str(count),'positive.npy'))
      negative_points = np.load(os.path.join(promptpath,i.split('.nii.gz')[0],str(count),'negative.npy'))
      input_points = np.concatenate([positive_points, negative_points])
      for m in range(len(input_points)):
        a = input_points[m][0]
        input_points[m][0] = input_points[m][1]
        input_points[m][1] = a
      input_labels = np.concatenate([np.ones(positive_points.shape[0]), np.zeros(negative_points.shape[0])])
      if (len(positive_points) == 0):
        count += 1
        continue
      if c == 0:
      _, out_obj_ids, out_mask_logits = sam2predictor.add_new_points_or_box(
        inference_state=inference_state,
        frame_idx=count,
        obj_id=1,
        points=input_point,
        labels=input_label,
          )

      mask = (out_mask_logits[0] > 0.0).cpu().numpy()
      gt = np.load(os.path.join(datapath,i,'binarymask',str(j) + '.npy'))
      dice = []
      iou = []
      count += 1
      os.makedirs(os.path.join(resultpath,i,str(j)),exist_ok=True)
      np.save(os.path.join(resultpath,i,str(j),'mask' + str(m)+'.npy'),mask)
      dice.append(compute_dice(gt,mask))
        # iou.append(compute_iou(masks[m],gt))
      print(dice)
      d = np.argsort(dice)
      np.save(os.path.join(resultpath,i,str(j),'dice.npy'),dice)
      # np.save(os.path.join(resultpath,i,j,k,l.split('.')[0],'iou.npy'),iou)

frame loading (JPEG): 100%|██████████| 249/249 [00:12<00:00, 20.43it/s]


/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/0


/content/drive/MyDrive/inference_SAM_video/segment-anything-2/sam2/sam2_video_predictor.py:873: UserWarning: /content/drive/MyDrive/inference_SAM_video/segment-anything-2/sam2/_C.so: undefined symbol: _ZN3c1015SmallVectorBaseIjE8grow_podEPKvmm

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/segment-anything-2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(


[0.9727428971760069]
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/1
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/2
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/3
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/4
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/5
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/6
[0.5632550765089144]
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/7
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/8
[0.0]
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/9
[0.0]
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/10
[0.0]
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/11
[0.0]
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/12
[4.301283395433112e-05]
/content/drive/MyDrive/inference_SAM_video/data/polypge

KeyboardInterrupt: 

In [ ]:
!git clone https://github.com/openmedlab/SAM-Med2D

Cloning into 'SAM-Med2D'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 89 (delta 22), reused 69 (delta 8), pack-reused 0
Receiving objects: 100% (89/89), 8.88 MiB | 16.16 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [ ]:
%cd /content/drive/MyDrive/inference_SAM_video/segment-anything
!pip install -e .

/content/drive/MyDrive/inference_SAM_video/segment-anything
Obtaining file:///content/drive/MyDrive/inference_SAM_video/segment-anything
  Preparing metadata (setup.py) ... done
  Running setup.py develop for segment_anything


In [ ]:
from segment_anything import SamPredictor, sam_model_registry
sam = sam_model_registry["vit_b"](checkpoint="/content/drive/MyDrive/inference_SAM_video/ckpt/sam_vit_b.pth")
sam.to(device='cuda')
predictor = SamPredictor(sam)

In [5]:
def repixel_value(arr, is_seg=False):
    if not is_seg:
        min_val = arr.min()
        max_val = arr.max()
        new_arr = (arr - min_val) / (max_val - min_val + 1e-10) * 255.
    return new_arr

In [ ]:
import os
import cv2
import nibabel
import torch
resultpath = '/content/drive/MyDrive/inference_SAM_video/result/Brats/sam_pointper103frame'
datapath = '/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr'
promptpath = '/content/drive/MyDrive/inference_SAM_video/prompt/point/Brats'
visualize = '/content/drive/MyDrive/inference_SAM_video/visresult'
# classes = ['bg_change','blood','low_brightness','regular','smoke']
# direction = ['left','right']
for i in os.listdir(datapath):
  img3D = nibabel.load(os.path.join(datapath,i)).get_fdata()[np.newaxis,:,:,:]
  mask = nibabel.load(os.path.join(datapath,i)).get_fdata()[np.newaxis,:,:,:]
  img3d = torch.from_numpy(img3D)
  count = 0
  for j in range(img3D.shape[3]):
      if os.path.exists(os.path.join(resultpath,i,str(j),'dice.npy')):
        continue
      print(os.path.join(datapath,i,str(j)))
      p = np.load(os.path.join(promptpath,i.split('.nii.gz')[0],str(count),'positive.npy'))
      if len(p) == 0:
        count += 1
        continue
      input_image = torch.repeat_interleave(img3d, repeats=3, dim=0)
      img2d,gt2d = repixel_value(input_image[:,:,:,j]),repixel_value(mask[:,:,:,j])
      gt2d = gt2d.astype(bool)
      img2d = img2d.permute(1,2,0).numpy()
      predictor.set_image(img2d)
      if count % 104 == 0:
        positive_points = np.load(os.path.join(promptpath,i.split('.nii.gz')[0],str(count),'positive.npy'))
        negative_points = np.load(os.path.join(promptpath,i.split('.nii.gz')[0],str(count),'negative.npy'))
        input_points = np.concatenate([positive_points, negative_points])
        for m in range(len(input_points)):
          a = input_points[m][0]
          input_points[m][0] = input_points[m][1]
          input_points[m][1] = a
        input_labels = np.concatenate([np.ones(positive_points.shape[0]), np.zeros(negative_points.shape[0])])
        # input_box = np.load(os.path.join(promptpath,i,k,l.split('.')[0],'bbox_prompt.npy'))[0]
        masks, _, prev_mask = predictor.predict(
          point_coords=input_points,
          point_labels=input_labels,
          box = None,
          multimask_output=True,
        )
      else:
        masks, _, prev_mask = predictor.predict(
          point_coords=input_points,
          point_labels=input_labels,
          box = None,
          mask_input = prev_mask,
          multimask_output=True,
        )
      count += 1
      os.makedirs(os.path.join(resultpath,i,str(j)),exist_ok=True)
      dice = []
      for m in range(masks.shape[0]):
        np.save(os.path.join(resultpath,i,str(j),'mask' + str(m)+'.npy'),masks[m])
        dice.append(compute_dice(gt2d,masks[m]))
        # iou.append(compute_iou(masks[m],gt))
      d = np.argsort(dice)
      prev_mask = prev_mask[[d[-1]]]
      np.save(os.path.join(resultpath,i,str(j),'dice.npy'),dice)
      # np.save(os.path.join(resultpath,i,j,k,l.split('.')[0],'iou.npy'),iou)

流式输出内容被截断，只能显示最后 5000 行内容。
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_00641.nii.gz/96
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_00641.nii.gz/97
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_00641.nii.gz/98
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_00641.nii.gz/99
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_00641.nii.gz/100
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_00641.nii.gz/101
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_00641.nii.gz/102
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_00641.nii.gz/103
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_00645.nii.gz/0
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_00645.nii.gz/1
/content/drive/MyDrive/lighting

In [ ]:
import os
import cv2
resultpath = '/content/drive/MyDrive/inference_SAM_video/result/polypgen/sam_pointperframe'
datapath = '/content/drive/MyDrive/inference_SAM_video/data/polypgen'
promptpath = '/content/drive/MyDrive/inference_SAM_video/prompt/point/polypgen'
visualize = '/content/drive/MyDrive/inference_SAM_video/visresult'
# classes = ['bg_change','blood','low_brightness','regular','smoke']
# direction = ['left','right']
for i in os.listdir(datapath):
  count = 0
  for j in range(len(os.listdir(os.path.join(datapath,i,'masks')))):
      # if os.path.exists(os.path.join(resultpath,i,j,k,l.split('.')[0],'dice.npy')):
      #   continue
      print(os.path.join(datapath,i,'masks',str(j)))
      img = cv2.imread(os.path.join(datapath,i,'images',str(j) + '.jpg'))
      p = np.load(os.path.join(promptpath,i,str(count),'positive.npy'))
      if len(p) == 0:
        count += 1
        continue
      print(img.shape)
      predictor.set_image(img)
      if count % 1 == 0:
        positive_points = np.load(os.path.join(promptpath,i,str(count),'positive.npy'))
        negative_points = np.load(os.path.join(promptpath,i,str(count),'negative.npy'))
        input_points = np.concatenate([positive_points, negative_points])
        for m in range(len(input_points)):
          a = input_points[m][0]
          input_points[m][0] = input_points[m][1]
          input_points[m][1] = a
        input_labels = np.concatenate([np.ones(positive_points.shape[0]), np.zeros(negative_points.shape[0])])
        # input_box = np.load(os.path.join(promptpath,i,k,l.split('.')[0],'bbox_prompt.npy'))[0]
        masks, _, prev_mask = predictor.predict(
          point_coords=input_points,
          point_labels=input_labels,
          box = None,
          multimask_output=True,
        )
      else:
        masks, _, prev_mask = predictor.predict(
          point_coords=input_points,
          point_labels=input_labels,
          box = None,
          mask_input = prev_mask,
          multimask_output=True,
        )
      count += 1
      gt = np.load(os.path.join(datapath,i,'binarymask',str(j) + '.npy'))
      dice = []
      iou = []
      os.makedirs(os.path.join(resultpath,i,str(j)),exist_ok=True)
      for m in range(masks.shape[0]):
        np.save(os.path.join(resultpath,i,str(j),'mask' + str(m)+'.npy'),masks[m])
        dice.append(compute_dice(gt,masks[m]))
        # iou.append(compute_iou(masks[m],gt))
      d = np.argsort(dice)
      prev_mask = prev_mask[[d[-1]]]
      np.save(os.path.join(resultpath,i,str(j),'dice.npy'),dice)
      # np.save(os.path.join(resultpath,i,j,k,l.split('.')[0],'iou.npy'),iou)

/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/0
(1064, 1440, 3)
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/1
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/2
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/3
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/4
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/5
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/6
(1064, 1440, 3)
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/7
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/8


KeyboardInterrupt: 

In [ ]:
import os
resultpath = '/content/drive/MyDrive/inference_SAM_video/result_sam_bbox300perframe'
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
classes = ['bg_change','blood','low_brightness','regular','smoke']
direction = ['left','right']
for j in classes:
  dice = []
  for k in direction:

    for i in os.listdir(resultpath):
      for l in os.listdir(os.path.join(resultpath,i,j,k)):
        if l == '.ipynb_checkpoints':
          continue
        # print(os.path.join(datapath,i,j,k,l))
        d = np.load(os.path.join(resultpath,i,j,k,l,'dice.npy'))
        dice.append(max(d))
        # IOU.append(max(np.load(os.path.join(resultpath,i,j,k,l,'iou.npy'))))
  print('average dice of ' + j + k + ' is ' + str(np.mean(dice)))
  # print('average iou of ' + j + k + ' is ' + str(np.mean(IOU)))
        # image = cv2.imread(os.path.join(datapath,i,j,k,l + '.png'))
        # mask = np.load(os.path.join(resultpath,i,j,k,l,'mask0.npy'))
        # print(image.shape)

        # gt = np.load(os.path.join(datapath,i,'ground_truth',k,l.split('.')[0] + '.npy'))
        # image[gt] = 13
        # image[mask] = 0
        # cv2_imshow(cv2.imread(os.path.join(datapath,i,j,k,l + '.png')))
        # point = np.load(os.path.join(promptp,i,k,l.split('.')[0],'positive.npy'))
        # for p in point:
        #   print(p)
        #   print(gt[p[0]][p[1]])
        #   cv2.circle(image, (int(p[1]), int(p[0])), 5, (0, 255, 0),thickness=-1)
        # cv2_imshow(image)
      # print('average dice of ' + i + j + k + ' is ' + str(np.mean(dice)))
      # print('average iou of ' + i + j + k + ' is ' + str(np.mean(IOU)))

average dice of bg_changeright is 0.6912629331678185
average dice of bloodright is 0.5662082263763284
average dice of low_brightnessright is 0.40933011891031607
average dice of regularright is 0.6122629458304449
average dice of smokeright is 0.5734609869479346


sam bbox per 10 frame is
average dice of bg_changeright is 0.7766800130947736
average dice of bloodright is 0.6269097334727318
average dice of low_brightnessright is 0.4414942287287958
average dice of regularright is 0.7113917040782672
average dice of smokeright is 0.666704752918767

sam bbox per frame
average dice of bg_changeright is 0.8156032569990948
average dice of bloodright is 0.6592212130857059
average dice of low_brightnessright is 0.4824622634145536
average dice of regularright is 0.7435796128012453
average dice of smokeright is 0.702584233428289

sam bbox per 300 frame
average dice of bg_changeright is 0.6912629331678185
average dice of bloodright is 0.5662082263763284
average dice of low_brightnessright is 0.40933011891031607
average dice of regularright is 0.6122629458304449
average dice of smokeright is 0.5734609869479346

prompt 10 per frame sam
average dice of bg_changeright is 0.8508227364865439
average dice of bloodright is 0.7866620446102477
average dice of low_brightnessright is 0.5440335243065363
average dice of regularright is 0.8387021200316954
average dice of smokeright is 0.8220985925576451

In [ ]:
a = np.zeros((1,1,1080,1920))
b = np.zeros((1,1,1080,1920))
c = [a,b]
d = np.concatenate(c)
print(d.shape)

(2, 1, 1080, 1920)


In [ ]:
sam_jointmask_path = '/content/drive/MyDrive/inference_SAM_video/sam_bboxper300frame'
resultpath = '/content/drive/MyDrive/inference_SAM_video/result_sam_bbox300perframe'
classes = ['bg_change','blood','low_brightness','regular','smoke']
direction = ['left','right']
import shutil
import os
import numpy as np
for j in classes:
  for i in os.listdir(resultpath):
    for k in direction:
      for l in range(len(os.listdir(os.path.join(resultpath,i,j,k)))):
        print(os.path.join(resultpath,i,j,k,str(l)))
        dice = np.load(os.path.join(resultpath,i,j,k,str(l),'dice.npy'))
        b = np.argsort(dice)
        os.makedirs(os.path.join(sam_jointmask_path,i,j,k,str(l)),exist_ok=True)
        shutil.move(os.path.join(resultpath,i,j,k,str(l),'mask' + str(b[-1]) + '.npy'),os.path.join(sam_jointmask_path,i,j,k,str(l),'mask.npy'))

流式输出内容被截断，只能显示最后 5000 行内容。
/content/drive/MyDrive/inference_SAM_video/result_sam_bbox300perframe/2/low_brightness/right/100
/content/drive/MyDrive/inference_SAM_video/result_sam_bbox300perframe/2/low_brightness/right/101
/content/drive/MyDrive/inference_SAM_video/result_sam_bbox300perframe/2/low_brightness/right/102
/content/drive/MyDrive/inference_SAM_video/result_sam_bbox300perframe/2/low_brightness/right/103
/content/drive/MyDrive/inference_SAM_video/result_sam_bbox300perframe/2/low_brightness/right/104
/content/drive/MyDrive/inference_SAM_video/result_sam_bbox300perframe/2/low_brightness/right/105
/content/drive/MyDrive/inference_SAM_video/result_sam_bbox300perframe/2/low_brightness/right/106
/content/drive/MyDrive/inference_SAM_video/result_sam_bbox300perframe/2/low_brightness/right/107
/content/drive/MyDrive/inference_SAM_video/result_sam_bbox300perframe/2/low_brightness/right/108
/content/drive/MyDrive/inference_SAM_video/result_sam_bbox300perframe/2/low_brightness/right/109
/co

next 2 is flip x y

average dice of bg_changeright is 0.14882881989370794
average dice of bloodright is 0.13072415882435398
average dice of low_brightnessright is 0.127663376589161
average dice of regularright is 0.12449286127611785
average dice of smokeright is 0.11316814240614181
THis is for prompt with frame 0

average dice of bg_changeright is 0.1298453919699352
average dice of bloodright is 0.11552156283667807
average dice of low_brightnessright is 0.10794421455616063
average dice of regularright is 0.10737336391847913
average dice of smokeright is 0.09900856586457903
this is for prompt along frame


sammed 2d prompt along
average dice of bg_changeright is 0.6859137698662229
average dice of bloodright is 0.5318488676771863
average dice of low_brightnessright is 0.4152789520159787
average dice of regularright is 0.6318540580455778
average dice of smokeright is 0.58022117064358

sammed2d prompt once is
average dice of bg_changeright is 0.56934240683007
average dice of bloodright is 0.44181135991572235
average dice of low_brightnessright is 0.39389298443107096
average dice of regularright is 0.5302526956790408
average dice of smokeright is 0.4928251604902058

In [ ]:
0.01/0.87

0.011494252873563218

In [ ]:
a = 0.7864913644226745
b =0.8410604998835951
print((a + b)/2)


0.8137759321531348


In [ ]:
import os
boxpromptpath = '/content/drive/MyDrive/inference_SAM_video/bbox_prompt'
data = '/content/drive/MyDrive/inference_SAM_video/9'
import cv2
import shutil
import numpy as np
from google.colab.patches import cv2_imshow
from PIL import Image
direction = ['left','right']
for i in os.listdir(data):
  for j in direction:
    for k in os.listdir(os.path.join(data,i,'ground_truth',j)):
      print(os.path.join(data,i,'ground_truth',j,k))
      gt = np.load(os.path.join(data,i,'ground_truth',j,k.split('.')[0] + '.npy'))
      bbox = generate_bbox(gt)
      os.makedirs(os.path.join(boxpromptpath,i,j,k.split('.')[0]),exist_ok=True)
      np.save(os.path.join(boxpromptpath,i,j,k.split('.')[0], 'bbox_prompt.npy'),bbox)

/content/drive/MyDrive/inference_SAM_video/9/2/ground_truth/left/131.npy
/content/drive/MyDrive/inference_SAM_video/9/2/ground_truth/left/143.npy
/content/drive/MyDrive/inference_SAM_video/9/2/ground_truth/left/10.npy
/content/drive/MyDrive/inference_SAM_video/9/2/ground_truth/left/146.npy
/content/drive/MyDrive/inference_SAM_video/9/2/ground_truth/left/138.npy
/content/drive/MyDrive/inference_SAM_video/9/2/ground_truth/left/113.npy
/content/drive/MyDrive/inference_SAM_video/9/2/ground_truth/left/120.npy
/content/drive/MyDrive/inference_SAM_video/9/2/ground_truth/left/1.npy
/content/drive/MyDrive/inference_SAM_video/9/2/ground_truth/left/147.npy
/content/drive/MyDrive/inference_SAM_video/9/2/ground_truth/left/104.npy
/content/drive/MyDrive/inference_SAM_video/9/2/ground_truth/left/132.npy
/content/drive/MyDrive/inference_SAM_video/9/2/ground_truth/left/107.npy
/content/drive/MyDrive/inference_SAM_video/9/2/ground_truth/left/14.npy
/content/drive/MyDrive/inference_SAM_video/9/2/ground_t

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def generate_bbox(input_mask):
  bbox = []
  x = []
  y = []
  for i in range(len(input_mask)):
    for j in range(len(input_mask[0])):
      if input_mask[i][j]:
        x.append(j)
        y.append(i)
  bbox = [[min(x),min(y),max(x),max(y)]]
  return bbox
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([251/255, 252/255, 30/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='blue', facecolor=(0,0,0,0), lw=2))

In [ ]:
import os
resultpath = '/content/drive/MyDrive/inference_SAM_video/result'
promptp = '/content/drive/MyDrive/inference_SAM_video/prompt_for_video_n'
oldprompt = '/content/drive/MyDrive/inference_SAM_video/prompt_for_video/9'
import cv2
import shutil
import numpy as np
from google.colab.patches import cv2_imshow
for i in os.listdir(promptp):
    for k in direction:
      dice = []
      IOU = []
      for l in os.listdir(os.path.join(datapath,i,'ground_truth',k)):
        if l == '.ipynb_checkpoints':
          continue
        # shutil.copy(os.path.join(oldprompt,i,k,l.split('.')[0],'negative.npy'),os.path.join(promptp,i,k,l.split('.')[0],'negative.npy'))
        # if i == '2':
        #   continue
        # print(os.path.join(datapath,i,k,l))
        # if os.path.exists(os.path.join(promptp,i,k,l.split('.')[0],'positive.npy')):
        #   continue
        # os.makedirs(os.path.join(promptp,i,k,l.split('.')[0]),exist_ok=True)
        # gt = np.load(os.path.join(datapath,i,'ground_truth',k,l.split('.')[0] + '.npy'))
        # mask = np.zeros(gt.shape)
        # mask[gt] = 2

        # # x = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        # # print(x.shape)
        # point = prompt_generate(mask)
        # np.save(os.path.join(promptp,i,k,l.split('.')[0],'positive.npy'),point)
      # print('average dice of ' + i + j + k + ' is ' + str(np.mean(dice)))
      # print('average iou of ' + i + j + k + ' is ' + str(np.mean(IOU)))

In [ ]:
import random
def prompt_generate(masks):
  positive_points = []
  x_axis = [[0,216],[216,432],[432,648],[648,864],[864,1080]]
  y_axis = [[0,384],[384,768],[768,1152],[1152,1536],[1536,1920]]
  max_value = []
  indexvalue = []
  position_value = []
  l = []
  for x in x_axis:
    for y in y_axis:
      maskp = mask[x[0]:x[1],y[0]:y[1]]
      _,m = cv2.threshold(maskp, 1, 1, cv2.THRESH_BINARY)
      m = m.astype(np.int8)
      c,labels,stas,center = cv2.connectedComponentsWithStats(m,8,cv2.CV_32S)
      if c == 1:
        continue
      max = 0
      index = 0
      for i in range(1,c):
        if stas[i][4] > max:
          max = stas[i][4]
          index = i
      l.append(labels)
      indexvalue.append(index)
      max_value.append(max)
      position_value.append([x[0],x[1],y[0],y[1]])
  p = np.argsort(max_value)
  p = p[::-1]
  for i in range(5):
    position = position_value[p[i]]
    index = indexvalue[p[i]]
    label = l[p[i]]
    po = []
    t = 40
    for x in range(0,len(label)):
      for y in range(0,len(label[0])):
        if label[x][y] == index:
          if x + t < len(label) and y + t < len(label[0]) and x-t > 0 and y-t>0 and con(label,x,y,index,t):
            po.append([x+position[0],y + position[2]])
    positive_points.append(random.choice(po))
  return positive_points

In [ ]:
def con(label,x,y,index,t):
  if label[x-t][y] == index and label[x+t][y] == index and label[x][y-t] == index and label[x][y+t] == index and label[x-t][y-t] == index and label[x-t][y+t] ==index and label[x+t][y-t]==index and label[x+t][y+t] == index:
    return True
  return False

In [ ]:
import random
def generate(mask):
  positive_points = []
  negative_points = []
  t = 1
  for x in range(0,mask.shape[0]):
    for y in range(0,mask.shape[1]):
      if mask[x][y] :
        if x + t < len(mask) and y + t < len(mask[0]) and x-t > 0 and y-t>0 and con(mask,x,y,1,t):
          positive_points.append([x,y])
      else:
        negative_points.append([x,y])
  if len(positive_points) == 0:
    return [],[]
  p = random.choices(positive_points,k = 5)
  n = random.choices(negative_points,k=5)
  return p,n

In [ ]:
#generate point prompt for polygen
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
datapath = '/content/drive/MyDrive/inference_SAM_video/data/polypgen'
promptpath = '/content/drive/MyDrive/inference_SAM_video/prompt/point/polypgen'
for i in os.listdir(datapath):
  for j in range(len(os.listdir(os.path.join(datapath,i,'masks')))):
    print(os.path.join(datapath,i,'masks',str(j) + '.jpg'))
    mask = cv2.imread(os.path.join(datapath,i,'masks',str(j) + '.jpg'),0)
    # for p in pos_points:
    #   cv2.circle(mask, (int(p[1]), int(p[0])), 5, (0, 255, 0),thickness=-1)
    m = mask.copy()
    m[m>20] = 1
    m[m!=1] = 0
    os.makedirs(os.path.join(datapath,i,'binarymask'),exist_ok=True)
    os.makedirs(os.path.join(promptpath,i,str(j)),exist_ok=True)
    np.save(os.path.join(datapath,i,'binarymask',str(j) + '.npy'),m)
    pos_points,neg_point = generate(m)
    np.save(os.path.join(promptpath,i,str(j),'positive.npy'),pos_points)
    np.save(os.path.join(promptpath,i,str(j),'negative.npy'),neg_point)
    # for p in pos_points:
    #    cv2.circle(mask, (int(p[1]), int(p[0])), 5, (0, 255, 0),thickness=-1)
    # cv2_imshow(mask)

/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/0.jpg
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/1.jpg
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/2.jpg
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/3.jpg
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/4.jpg
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/5.jpg
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/6.jpg
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/7.jpg
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/8.jpg
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/9.jpg
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/10.jpg
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/11.jpg
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/12.jpg
/content/drive/MyDrive

In [ ]:
#generate prompt slice by slice
import nibabel
import os
import numpy as np
import matplotlib.pyplot as plt
datapath = '/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr'
promptpath = '/content/drive/MyDrive/inference_SAM_video/prompt/point/Brats'
for i in os.listdir(datapath):
  image = nibabel.load(os.path.join(datapath,i))
  image = image.get_fdata()
  mask = nibabel.load(os.path.join('/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/labelsTr',i))
  mask = mask.get_fdata()
  for j in range(image.shape[2]):
    print(os.path.join(datapath,i,str(j)))
    if os.path.exists(os.path.join(promptpath,i.split('.nii.gz')[0],str(j),'positive.npy')) and os.path.exists(os.path.join(promptpath,i.split('.nii.gz')[0],str(j),'negative.npy')):
      continue
    os.makedirs(os.path.join(promptpath,i.split('.nii.gz')[0],str(j)),exist_ok=True)
    positive,negative = generate(mask[:,:,j])
    # plt.imshow(mask[:,:,j])
    # for p in positive:
    #   plt.scatter(p[1],p[0])
    # plt.show()
    np.save(os.path.join(promptpath,i.split('.nii.gz')[0],str(j),'positive.npy'),positive)
    np.save(os.path.join(promptpath,i.split('.nii.gz')[0],str(j),'negative.npy'),negative)

流式输出内容被截断，只能显示最后 5000 行内容。
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_00641.nii.gz/96
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_00641.nii.gz/97
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_00641.nii.gz/98
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_00641.nii.gz/99
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_00641.nii.gz/100
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_00641.nii.gz/101
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_00641.nii.gz/102
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_00641.nii.gz/103
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_00645.nii.gz/0
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_00645.nii.gz/1
/content/drive/MyDrive/lighting

In [ ]:
import nibabel
import os
datapath = '/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr'
for i in os.listdir(datapath):
  a = nibabel.load(os.path.join(datapath,i))
  print(a.shape)

(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 160, 104)
(160, 16